## Install ultralytics library

In [ ]:
pip install ultralytics

## Import Libraries

In [ ]:
import os
import numpy as np
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from PIL import Image  # PIL is used to load the image
import matplotlib.pyplot as plt
import cv2


from ultralytics import YOLO
from pathlib import Path

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Display an image to test if the mount is successful

In [ ]:
dataset_path = "/content/drive/MyDrive/datasets/catsanddogs"
image_dir = os.path.join(dataset_path, "images")
# label_dir = os.path.join(dataset_path, "labels")

In [ ]:
def displayImage(image_path):
  # Path to your image file in Google Drive
  image = Image.open(image_path)

  # Display the image using matplotlib
  plt.imshow(image)
  plt.axis('off')  # Hide axes for cleaner display
  plt.show()

In [ ]:
displayImage('/content/drive/My Drive/datasets/catsanddogs/images/cat.0.jpg')

## Check if GPU is used

In [ ]:
# Check if TensorFlow can access the GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print(f"TensorFlow is using GPU: {physical_devices[0]}")
else:
    print("No GPU available.")

## Define the image size

In [ ]:
SIZE = 640
image_size = (SIZE, SIZE)

## Feed the images to YOLO

In [ ]:
IMAGE_DIR = Path(image_dir) # Folder with your images
label_dir = ("labels") # Where to save YOLO labels
model_name = "yolov8x.pt"         # Try yolov8m.pt or yolov8x.pt for better accuracy

# Load YOLO model
model = YOLO(model_name)

# Run inference
results = model(source=str(IMAGE_DIR), save_txt=True, project=str(label_dir), name=".")


## Verify the if the bounding boxes generated by YOLO are correct for each image

In [ ]:
LABEL_DIR = 'labels/labels'
IMG_SIZE = (640, 640)  # Resize if needed

def load_and_display(image_filename):
    image_path = os.path.join(IMAGE_DIR, image_filename)
    label_path = os.path.join(LABEL_DIR, os.path.splitext(image_filename)[0] + ".txt")

    img = cv2.imread(image_path)
    img = cv2.resize(img, IMG_SIZE)

    imageHeight, imageWidth, _ = img.shape

    # Draw boxes
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                class_id, x_center, y_center, width, height = map(float, parts)

                # Convert from YOLO format to pixel coordinates
                x_center *= imageWidth
                y_center *= imageHeight
                width *= imageWidth
                height *= imageHeight

                x_min = int(x_center - width / 2)
                y_min = int(y_center - height / 2)
                x_max = int(x_center + width / 2)
                y_max = int(y_center + height / 2)

                # Draw rectangle and class label
                cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cv2.putText(img, f"Class {int(class_id)}", (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Convert BGR to RGB for matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title(image_filename)
    plt.show()

# Iterate through all images and display
for image_file in os.listdir(IMAGE_DIR):
    if image_file.lower().endswith(('.jpg', '.png', '.jpeg')):
        load_and_display(image_file)
